In [0]:
import requests
from datetime import datetime
import logging
import urllib3

import sailpoint
import sailpoint.v2024
from sailpoint.configuration import (Configuration, ConfigurationParams)
from sailpoint.paginator import Paginator

from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, ArrayType, MapType 

In [0]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create console handler and set level to info
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# Create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Add formatter to ch
ch.setFormatter(formatter)

# Add ch to logger
logger.addHandler(ch)

In [0]:
tenant = 'devrel-ga-5668'
client_name = 'sailpoint-readall-client'
secret_name = 'sailpoint-readall-secret'

In [0]:
logger.info('Defining API credentials')
api_url = f'https://{tenant}.api.identitynow-demo.com'
token_url = f'https://{tenant}.api.identitynow-demo.com/oauth/token'
client_id = dbutils.secrets.get(scope='sailpoint-devrel', key=client_name)
client_secret = dbutils.secrets.get(scope='sailpoint-devrel', key=secret_name)

In [0]:
logger.info('Configuring SailPoint SDK')
configurationParams = ConfigurationParams()
configurationParams.base_url = api_url
configurationParams.token_url = token_url
configurationParams.client_id = client_id
configurationParams.client_secret = client_secret
configuration = Configuration(configurationParams)
configuration.experimental = True
configuration.retries = urllib3.Retry(
    total=5,  # Total number of retries
    backoff_factor=10,  # Backoff factor for retry delay
    status_forcelist=[429, 500, 502, 503, 504],  # HTTP status codes to retry on
    allowed_methods=["GET"]  # HTTP methods to retry
)

In [0]:
object_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("type", StringType(), True),
    ]
)

In [0]:
entitlement_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("displayName", StringType(), True),
        StructField("created", StringType(), True),
        StructField("modified", StringType(), True),
        StructField("value", StringType(), True),
        StructField("source", object_schema, True),
    ]
)

In [0]:
async def get_all_ent_count():
    with sailpoint.v2024.ApiClient(configuration) as api_client:
        search = '''
            {
                "indices": [
                    "entitlements"
                ],
                "query": {
                    "query": "*"
                }
            }
        ''' # search
        new_search = sailpoint.v2024.models.search.Search.from_json(search)
        search_instance = sailpoint.v2024.SearchApi(api_client)
        response = search_instance.search_post_with_http_info(search=new_search, offset=0, limit=1, count=True)
        return response.headers['X-Total-Count']

async def get_all_ents(limit):
    entitlements = []
    with sailpoint.v2024.ApiClient(configuration) as api_client:
        search = sailpoint.v2024.Search(
            indices=['entitlements'], 
            query=sailpoint.v2024.Query(query='*'), 
            sort=['-name'])
        try:
            entitlements = Paginator.paginate_search(sailpoint.v2024.SearchApi(api_client), search, increment=100, limit=1000)
            logger.info(f'Retrieved {entitlements.count} entitlements')

        except Exception as e:
            logger.error("Exception when calling paginate_search: %s\n" % e)
            
    return entitlements

In [0]:
filter = None
total_entitlement_count = int(await get_all_ent_count())
logger.info(f'Total entitlements: {total_entitlement_count}')
all_entitlements = await get_all_ents(total_entitlement_count)
logger.info(f'Retrieved {len(all_entitlements)} entitlements')

In [0]:
try:
    logger.info('Creating data frame...')
    ents_df = spark.createDataFrame(all_entitlements, entitlement_schema)
    today = datetime.utcnow().date()
    ents_df = ents_df.withColumn('day', lit(today))

    logger.info('Writing to delta...')
    ents_df.write \
        .format('delta') \
        .mode('append') \
        .partitionBy('day') \
        .saveAsTable(f'`dunker_databricks_space`.sailpoint.entitlements')

except Exception as e:
    logger.error(f'Error in creating data frame or writing to delta: {e}')